In [1]:
# Chạy cell này đầu tiên để cài đặt các thư viện cần thiết.
!pip install pandas nltk scikit-learn gensim openai

In [2]:
# CELL 2: IMPORT THƯ VIỆN VÀ NẠP DỮ LIỆU
# ===================================================================
import pandas as pd
import re
import numpy as np
import itertools
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Tải tài nguyên cần thiết cho VADER
nltk.download('vader_lexicon')

# Tải dữ liệu "vàng" đã được xử lý và gộp từ nguồn UCSD
# Đây là file output từ notebook tích hợp dữ liệu của bạn.
try:
    df = pd.read_csv('ucsd_delaware_reviews_combined.csv')
    print("Tải dữ liệu nâng cấp thành công!")
    print(f"Tổng số review để xử lý: {len(df)}")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file 'ucsd_delaware_reviews_combined.csv'.")
    print("Hãy chắc chắn bạn đã chạy notebook tích hợp dữ liệu và file này tồn tại.")

# Hiển thị thông tin cơ bản để kiểm tra
if 'df' in locals():
    print(df.info())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Tải dữ liệu nâng cấp thành công!
Tổng số review để xử lý: 58289
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58289 entries, 0 to 58288
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gmap_id         58289 non-null  object 
 1   user_id         58289 non-null  object 
 2   user_name       58289 non-null  object 
 3   time            58289 non-null  int64  
 4   rating          58289 non-null  int64  
 5   text            58288 non-null  object 
 6   text_clean      58209 non-null  object 
 7   place_name      58289 non-null  object 
 8   address         58109 non-null  object 
 9   category        58288 non-null  object 
 10  avg_rating      58289 non-null  float64
 11  num_of_reviews  58289 non-null  int64  
 12  price           23279 non-null  object 
dtypes: float64(1), int64(3), object(9)
memory usage: 5.8+ MB
None


In [3]:
# CELL 3: FEATURE ENGINEERING - CÁC ĐẶC TRƯNG CƠ BẢN
# ===================================================================
print("Bắt đầu tạo các đặc trưng cơ bản...")

# 1. Tính toán điểm sentiment
analyzer = SentimentIntensityAnalyzer()
df.dropna(subset=['text'], inplace=True) # Đảm bảo không có text rỗng
df['sentiment_score'] = df['text'].apply(lambda text: analyzer.polarity_scores(str(text))['compound'])

# 2. Đặc trưng Metadata
df['review_length'] = df['text'].str.len()
df['user_review_count'] = df.groupby('user_id')['user_id'].transform('count')
df['rating_deviation'] = (df['rating'] - df['avg_rating']).fillna(0)

# 3. Suy luận Tín hiệu "Đã ghé thăm"
visit_keywords = [
    'visited', 'went to', 'ate here', 'dined here', 'was there',
    'stayed at', 'my visit', 'our visit', 'ordered', 'tried the'
]
df['has_visit_keyword'] = df['text'].str.contains('|'.join(visit_keywords), case=False, na=False)

print("Hoàn thành tạo các đặc trưng cơ bản.")
print(df[['user_name', 'sentiment_score', 'review_length', 'user_review_count', 'has_visit_keyword']].head())

Bắt đầu tạo các đặc trưng cơ bản...
Hoàn thành tạo các đặc trưng cơ bản.
        user_name  sentiment_score  review_length  user_review_count  \
0  Heather Carper           0.8176            112                  4   
1  Heather Carper           0.8176            112                  4   
2  STACY CLAVETTE           0.9685            283                  5   
3  STACY CLAVETTE           0.9685            283                  5   
4       Zion Hood           0.7269             74                  5   

   has_visit_keyword  
0              False  
1              False  
2              False  
3              False  
4              False  


In [4]:
# CELL 4: FEATURE ENGINEERING - NLP NÂNG CAO (TF-IDF & LDA)
# ===================================================================
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim.models import LdaModel

print("Bắt đầu các bước NLP nâng cao (có thể mất vài phút)...")

# --- a. Keyword Extraction (TF-IDF) ---
df['text_clean'] = df['text_clean'].fillna('')
tfidf_vectorizer = TfidfVectorizer(max_features=2000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_clean'])
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

def extract_top_keywords(doc, top_n=5):
    tfidf_vector = tfidf_vectorizer.transform([doc])
    sorted_indices = np.argsort(tfidf_vector.toarray()).flatten()[::-1]
    top_keywords = feature_names[sorted_indices[:top_n]]
    return ', '.join(top_keywords)

df['keywords'] = df['text_clean'].apply(extract_top_keywords)
print("-> Hoàn thành trích xuất từ khóa (Keywords).")

# --- b. Topic Modeling (LDA) ---
tokenized_data = [text.split() for text in df['text_clean']]
dictionary = corpora.Dictionary(tokenized_data)
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=10, random_state=42)

print("\nCác chủ đề được phát hiện bởi LDA:")
for idx, topic in lda_model.print_topics(-1):
    print(f'Topic {idx}: {topic}')

def get_dominant_topic(doc):
    bow = dictionary.doc2bow(doc.split())
    topics = lda_model.get_document_topics(bow, minimum_probability=0.0)
    dominant_topic = sorted(topics, key=lambda x: x[1], reverse=True)[0][0]
    return dominant_topic

df['topic'] = df['text_clean'].apply(get_dominant_topic)
print("\n-> Hoàn thành gán chủ đề (Topics).")
print(df[['text', 'keywords', 'topic']].head())

Bắt đầu các bước NLP nâng cao (có thể mất vài phút)...
-> Hoàn thành trích xuất từ khóa (Keywords).

Các chủ đề được phát hiện bởi LDA:
Topic 0: 0.048*"the" + 0.035*"to" + 0.024*"you" + 0.023*"i" + 0.023*"they" + 0.023*"and" + 0.023*"a" + 0.019*"is" + 0.018*"have" + 0.014*"of"
Topic 1: 0.051*"a" + 0.034*"to" + 0.031*"of" + 0.031*"place" + 0.030*"great" + 0.029*"for" + 0.025*"and" + 0.025*"nice" + 0.016*"good" + 0.013*"very"
Topic 2: 0.093*"the" + 0.039*"and" + 0.032*"was" + 0.029*"food" + 0.024*"is" + 0.021*"it" + 0.019*"best" + 0.017*"good" + 0.015*"love" + 0.014*"a"
Topic 3: 0.097*"and" + 0.090*"great" + 0.061*"very" + 0.055*"service" + 0.044*"friendly" + 0.043*"staff" + 0.042*"good" + 0.026*"food" + 0.021*"helpful" + 0.021*"always"
Topic 4: 0.046*"i" + 0.041*"and" + 0.038*"the" + 0.037*"was" + 0.033*"to" + 0.029*"my" + 0.025*"a" + 0.016*"me" + 0.016*"for" + 0.014*"it"

-> Hoàn thành gán chủ đề (Topics).
                                                text  \
0  Lived here for 3 year

In [5]:
# CELL 5: FEATURE ENGINEERING - ĐẶC TRƯNG THỜI GIAN
# ===================================================================
print("Bắt đầu tạo đặc trưng thời gian...")
df['datetime'] = pd.to_datetime(df['time'], unit='ms')
df = df.sort_values(by=['user_id', 'datetime'])
df['time_delta_seconds'] = df.groupby('user_id')['datetime'].diff().dt.total_seconds().fillna(0)
print("-> Hoàn thành tính chênh lệch thời gian (Timestamp Delta).")
print(df[['user_name', 'datetime', 'time_delta_seconds']].head())

Bắt đầu tạo đặc trưng thời gian...
-> Hoàn thành tính chênh lệch thời gian (Timestamp Delta).
          user_name                datetime  time_delta_seconds
648     Ronald Keys 2018-03-01 15:56:48.891        0.000000e+00
44104  Don Kelleher 2020-05-21 21:43:37.803        0.000000e+00
14627   Tom Carroll 2020-02-08 01:04:21.369        0.000000e+00
23922   Tom Carroll 2020-10-11 14:24:50.518        2.130243e+07
35287  Jeff Peacock 2017-07-23 02:04:50.769        0.000000e+00


In [6]:
# CELL 6: MODULE CHÍNH SÁCH - PHÁT HIỆN QUẢNG CÁO VÀ TẠO NHÃN ĐA TRỊ
# ===================================================================
print("Bắt đầu xây dựng Module Chính sách...")

# 1. Thêm dữ liệu giả để kiểm tra logic phát hiện URL
promo_text_1 = "Great place! visit www.mypromo.com for a 10% discount!"
df.loc[len(df)] = df.iloc[0]
df.loc[df.index[-1], 'text'] = promo_text_1

# 2. Logic phát hiện URL
url_pattern = r'(https|http|www)[^\s]+'
df['has_url'] = df['text'].str.contains(url_pattern, case=False, na=False)
print(f"-> Đã tìm thấy {df['has_url'].sum()} review chứa URL.")

# 3. Hàm tạo nhãn đa trị (multi-label) dựa trên luật
def create_multilabels(row):
    labels = []
    if row['sentiment_score'] < -0.5 and not row['has_visit_keyword']:
        labels.append('rant_no_visit')
    if row['has_url']:
        labels.append('ad')
    # Giả sử sau khi xem LDA, bạn thấy topic 1 là không liên quan (ví dụ)
    # Bạn cần tự điều chỉnh chỉ số topic này sau khi xem kết quả ở cell trên
    if row['topic'] == 1:
        labels.append('irrelevant')
    if not labels:
        labels.append('clean')
    return labels

df['multilabels'] = df.apply(create_multilabels, axis=1)
print("-> Hoàn thành tạo nhãn đa trị (multi-label) dựa trên luật.")
print("\nKiểm tra review quảng cáo giả vừa thêm:")
print(df[df['has_url']][['text', 'multilabels']])

Bắt đầu xây dựng Module Chính sách...


/tmp/ipython-input-1169806186.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['has_url'] = df['text'].str.contains(url_pattern, case=False, na=False)


-> Đã tìm thấy 7 review chứa URL.
-> Hoàn thành tạo nhãn đa trị (multi-label) dựa trên luật.

Kiểm tra review quảng cáo giả vừa thêm:
                                                    text          multilabels
9783   Incredibly delicious!!\nAs always.\nLuv the su...                 [ad]
31808  Very disorganized staff unfriendly and unhelpf...  [rant_no_visit, ad]
27067  I live by this place. I don't eat Chinese but ...                 [ad]
23844  (Translated by Google) Awwweeeessssooooomeeee ...     [ad, irrelevant]
52771  (Translated by Google) Slowwwww at the pharmac...     [ad, irrelevant]
37895  Time well spent and well worth it.\nEvery wher...                 [ad]
36516  Great place! visit www.mypromo.com for a 10% d...                 [ad]


In [7]:
# CELL 7: MODEL DEVELOPMENT - KẾ HOẠCH A: THỬ VỚI OPENAI API (ĐÃ SỬA LỖI)
# ===================================================================
import openai
from google.colab import userdata
import json
import time

print("Bắt đầu Kế hoạch A: Thử nghiệm với OpenAI API...")

# 1. Khởi tạo Client
try:
    api_key = userdata.get('OPENAI_API_KEY')
    client = openai.OpenAI(api_key=api_key)
    print("-> Đã kết nối với API của OpenAI thành công!")
    openai_available = True
except Exception as e:
    print(f"-> Lỗi kết nối OpenAI: {e}. Sẽ chuyển sang Kế hoạch B.")
    openai_available = False

# 2. Định nghĩa hàm phân loại (ĐÃ SỬA LỖI GỌI HÀM)
def classify_review_with_openai(review_text, category):
    system_prompt = """
    You are an AI assistant for Google Maps. Your task is to analyze a review and classify it based on quality policies.
    A review can have one or more violation labels. If no violations are found, classify it as "clean".

    Possible Labels:
    - "ad": Contains advertisements, promotions, or external links.
    - "irrelevant": The content is not related to the given category.
    - "rant_no_visit": A strong complaint that shows no evidence of a real visit.

    You must respond ONLY with a valid JSON object containing a single key "labels" which is a list of strings.
    For example: {"labels": ["clean"]} or {"labels": ["ad", "irrelevant"]}.
    """

    user_prompt = f"""
    Please classify the following review for a place in the category "{category}".
    Review Text: "{review_text}"
    """

    try:
        response = client.chat.completions.create(
            # SỬA LỖI Ở ĐÂY: Thêm 'model='
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"},
            temperature=0.1
        )
        return json.loads(response.choices[0].message.content)

    except Exception as e:
        return {"labels": [f"error_{str(e)}"]}

# 3. Chạy thử nghiệm nếu API khả dụng
if openai_available:
    print("\nHàm phân loại bằng OpenAI đã sẵn sàng. Bắt đầu chạy thử nghiệm...")
    # Lấy 10 dòng cuối để test, bao gồm cả review quảng cáo giả
    sample_df_openai = df.tail(10).copy()

    openai_results = sample_df_openai.apply(
        lambda row: classify_review_with_openai(row['text'], row['category']),
        axis=1
    )

    sample_df_openai['openai_labels'] = [res.get('labels', ['error']) for res in openai_results]

    print("\n-> Kết quả phân loại đa nhãn từ OpenAI (GPT-3.5-Turbo):")
    print(sample_df_openai[['text', 'multilabels', 'openai_labels']])
else:
    print("\nBỏ qua thử nghiệm OpenAI do kết nối không thành công.")

Bắt đầu Kế hoạch A: Thử nghiệm với OpenAI API...
-> Đã kết nối với API của OpenAI thành công!

Hàm phân loại bằng OpenAI đã sẵn sàng. Bắt đầu chạy thử nghiệm...

-> Kết quả phân loại đa nhãn từ OpenAI (GPT-3.5-Turbo):
                                                    text   multilabels  \
25777  First time I tried their food will say it wasn...       [clean]   
39737  I have been going here for years. All the staf...       [clean]   
35036                                            Awesome       [clean]   
1534                              Very helpful and nice.       [clean]   
42774  A nice beach!  Fairly busy, arrive early to ge...  [irrelevant]   
42827  A nice beach!  Fairly busy, arrive early to ge...  [irrelevant]   
22297  Good food, better if you eat seafood.  Friendl...       [clean]   
22299  Good food, better if you eat seafood.  Friendl...       [clean]   
19157                 Food was good, staff was friendly.       [clean]   
36516  Great place! visit www.mypromo.com 

In [8]:
# CELL 8: MODEL DEVELOPMENT - KẾ HOẠCH B: FALLBACK VỚI SCIKIT-LEARN
# ===================================================================
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

print("Thực hiện Kế hoạch B: Huấn luyện mô hình Scikit-learn.")

# 1. Chuẩn bị nhãn đơn để huấn luyện
def create_single_label(row):
    if 'ad' in row['multilabels']: return 'ad'
    if 'rant_no_visit' in row['multilabels']: return 'rant_no_visit'
    if 'irrelevant' in row['multilabels']: return 'irrelevant'
    return 'clean'

df['final_label'] = df.apply(create_single_label, axis=1)

# 2. Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
    df['text_clean'], df['final_label'],
    test_size=0.2, random_state=42, stratify=df['final_label']
)

# 3. Xây dựng và Huấn luyện Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=2000, stop_words='english')),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])
pipeline.fit(X_train, y_train)
print("-> Huấn luyện mô hình Scikit-learn hoàn tất!")

# 4. Dự đoán và lưu kết quả
df['sklearn_classification'] = pipeline.predict(df['text_clean'])
print("\n-> Kết quả phân loại từ Scikit-learn:")
print(df[['text', 'final_label', 'sklearn_classification']].head())

Thực hiện Kế hoạch B: Huấn luyện mô hình Scikit-learn.
-> Huấn luyện mô hình Scikit-learn hoàn tất!

-> Kết quả phân loại từ Scikit-learn:
                                                    text    final_label  \
648                     Great job have to have driver co          clean   
44104                             Very easy to work with          clean   
14627  Got the best haircut I've had in years by Marcus.          clean   
23922  Great service- courteous and efficient.  And o...          clean   
35287                                    low low prices!  rant_no_visit   

      sklearn_classification  
648                    clean  
44104                  clean  
14627                  clean  
23922                  clean  
35287          rant_no_visit  


In [9]:
# CELL 9: LƯU OUTPUT CUỐI CÙNG CHO DAY 3
# ===================================================================
# Lưu DataFrame đã được làm giàu với tất cả các feature và nhãn
final_output_filename = 'final_augmented_reviews_for_day3.csv'
df.to_csv(final_output_filename, index=False)

print(f"\nĐÃ HOÀN THÀNH DAY 2!")
print(f"Đã lưu thành công DataFrame cuối cùng vào file '{final_output_filename}'.")
print("File này chứa tất cả các đặc trưng và nhãn bạn cần cho Day 3 - Đánh giá mô hình.")


ĐÃ HOÀN THÀNH DAY 2!
Đã lưu thành công DataFrame cuối cùng vào file 'final_augmented_reviews_for_day3.csv'.
File này chứa tất cả các đặc trưng và nhãn bạn cần cho Day 3 - Đánh giá mô hình.
